In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
from itertools import permutations

Load data  
Encoding utf-8 can not be enough to encoding so cp1252 is suitable solution

In [2]:
market_sale = pd.read_csv('MarketSales.csv', encoding='cp1252')
market_sale.head()

C:\Users\vient\Anaconda3\envs\plot_env\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,BRANCHNR,...,CLIENTCODE,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,GENDER
0,11738.0,5863.0,SPRITE 1 LT LIMON AROMALI GAZOZ,18456,07/01/2017 0:00,1.0,2.00,2.00,1.85,52.0,...,467369,Sercan KIZILOK,156,SPR?TE,?ÇECEK,GAZLI ?ÇECEK,GAZOZ,08/01/2017 16:16,08/01/2017 16:17,E
1,10537.0,8.0,TOZ SEKER,18105,06/01/2017 0:00,5.0,2.65,13.25,12.27,8.0,...,131464,?smet ?INGIR,NaN,NaN,?ÇECEK,ÇAY KAHVE,SEKER TATLANDIRICI,07/01/2017 11:04,07/01/2017 11:05,E
2,11335.0,5979.0,FALIM SAKIZ 5LI NANE,18350,03/01/2017 0:00,1.0,0.40,0.40,0.37,40.0,...,656969,Ya??z KUBAL,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,04/01/2017 14:00,04/01/2017 14:01,E
3,11336.0,5979.0,FALIM SAKIZ 5LI NANE,18350,03/01/2017 0:00,1.0,0.40,0.40,0.37,40.0,...,656969,Ya??z KUBAL,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,04/01/2017 14:00,04/01/2017 14:01,E
4,10115.0,12808.0,FALIM SAKIZ 5LI CILEK,18005,05/01/2017 0:00,1.0,0.40,0.40,0.37,41.0,...,447336,Habibe AYSAN,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,06/01/2017 14:00,06/01/2017 14:01,K


### Apriori Algorithm  
In this file, I use Apriori Algorithm to find which goods were bought together  
Also I will find which shelf should be arraged closely for increasing market sale  
For more information, you can read this blog about Apriori: https://pbpython.com/market-basket-analysis.html

#### First, use Apriori for shelf

In [3]:
selected_col = ['DATE_', 'FICHENO','CATEGORY_NAME1']
data = market_sale[selected_col]
data.head()

,DATE_,FICHENO,CATEGORY_NAME1
0,07/01/2017 0:00,18456,?ÇECEK
1,06/01/2017 0:00,18105,?ÇECEK
2,03/01/2017 0:00,18350,GIDA
3,03/01/2017 0:00,18350,GIDA
4,05/01/2017 0:00,18005,GIDA


Checking null data and cleasing

In [4]:
data.isnull().sum()

DATE_                1
FICHENO              0
CATEGORY_NAME1    7167
dtype: int64

In [5]:
data = data.dropna(axis = 0)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 603941 entries, 0 to 611107
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   DATE_           603941 non-null  object
 1   FICHENO         603941 non-null  int64 
 2   CATEGORY_NAME1  603941 non-null  object
dtypes: int64(1), object(2)
memory usage: 18.4+ MB


In [7]:
data.value_counts('CATEGORY_NAME1')

CATEGORY_NAME1
GIDA                 231973
MEYVE SEBZE           90262
SÜT KAHVALTILIK       85492
?ÇECEK                73264
DETERJAN TEM?ZL?K     22748
KA?IT                 21577
KOZMET?K              20916
ET TAVUK              19447
S?GARA                19031
EV                    12816
BEBEK                  6382
PET                      33
dtype: int64

pd.crosstab :  computes a frequency table  
https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html

In [8]:
transaction = pd.crosstab(index = data['FICHENO'], columns = data['CATEGORY_NAME1'])

In [9]:
transaction.head()

CATEGORY_NAME1,?ÇECEK,BEBEK,DETERJAN TEM?ZL?K,ET TAVUK,EV,GIDA,KA?IT,KOZMET?K,MEYVE SEBZE,PET,S?GARA,SÜT KAHVALTILIK
FICHENO,,,,,,,,,,,,
15560,1,0,2,0,0,3,0,0,0,0,0,1
15561,1,0,2,0,0,3,0,0,0,0,0,1
15562,1,0,0,0,0,1,0,0,0,0,1,0
15563,1,0,0,0,0,0,0,0,2,0,1,0
15564,2,0,0,0,0,0,0,0,0,0,0,0


In [10]:
transaction.shape

(141783, 12)

In [11]:
def apriori(data, min_support = 0.04, max_length =3):
    # Step 1:
    # Creating a dictionary to stored support of an itemset.
    support = {} 
    L = [i for i in data.columns]
    
    # Step 2: 
    #generating combination of items with len i in ith iteration
    for i in range(1, max_length+1):
        #Input : arr[] = [1, 2, 3, 4],  
            #r = 2, with combination = 2
        #Output : [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
        # Ket hop cac cot data voi nhau
        c = set(combinations(L,i))
        
    # Reset "L" for next ith iteration
        L =set()     
    # Step 3: 
        #iterate through each item in "c"
        for j in c:
            #product: combination 2 array in pair
            #Input : arr1 = [10, 12] 
            #arr2 = [8, 9, 10] 
            #Output : [(10, 8), (10, 9), (10, 10), (12, 8), (12, 9), (12, 10)] 
            sup = data.loc[:,j].product(axis=1).sum()/len(data.index)
            #data.loc[:,j] - chọn tất cả các dòng trong data có chứa cột j
            #data.loc[:,j].product(axis=1): kết hợp các cột j lại, nếu tất cả các cột j đều > 0 thì tính là 1
            if sup > min_support:
                #print(sup, j)
                support[j] = sup
                
                # Appending frequent itemset in list "L", already reset list "L" 
                L = [i for i in (set(L) | set(j))]
        
    # Step 4: data frame with cols "items", 'support'
    result = pd.DataFrame([i for i in (support.items())], columns = ["Items", "Support"])
    return(result)

In [12]:
frequent_item = apriori(transaction, min_support = 0.1, max_length = 3)

In [13]:
frequent_item

,Items,Support
0,"(SÜT KAHVALTILIK,)",0.602978
1,"(S?GARA,)",0.134226
2,"(KA?IT,)",0.152183
3,"(KOZMET?K,)",0.147521
4,"(GIDA,)",1.636113
...,...,...
80,"(MEYVE SEBZE, GIDA, SÜT KAHVALTILIK)",5.873694
81,"(KOZMET?K, ?ÇECEK, GIDA)",1.003153
82,"(MEYVE SEBZE, GIDA, S?GARA)",0.221247
83,"(KOZMET?K, ?ÇECEK, DETERJAN TEM?ZL?K)",0.188789


In [14]:
def associate_rule(df, min_threshold=0.5):
    from itertools import permutations
    
    # STEP 1:
    #creating required varaible
    support = pd.Series(df.Support.values, index=df.Items).to_dict()
    data = []
    L= df.Items.values
    
    # Step 2:
    #generating rule using permutation
    p = [i for i in (permutations(L, 2))]
    #n = []
    #for i in range(len(p)):
    #    if not set(p[i][0]).issubset(p[i][1]):
     #       n.append(p[i])'''
    
    # Iterating through each rule
    for i in p:
        # If LHS(Antecedent) of rule is subset of RHS then valid rule.
        if set(i[0]).issubset(i[1]): 
            conf = support[i[1]]/support[i[0]]
            #print(i, conf)
            if conf > min_threshold:
                #print(i, conf)
                j = i[1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                leverage = support[i[1]] - (support[i[0]]* support[(j,)])
                convection = (1 - support[(j,)])/(1- conf)
                data.append([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage, convection])

         
    # STEP 3:
    result = pd.DataFrame(data, columns = ["antecedents", "consequents", "antecedent support", "consequent support",
                                        "support", "confidence", "Lift", "Leverage", "Convection"])
    return(result)

In [15]:
result = associate_rule(frequent_item, min_threshold=0.5)

In [16]:
sort_value = result.sort_values(by='Lift', ascending=False)
sort_value.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
190,"(?ÇECEK, DETERJAN TEM?ZL?K)","(DETERJAN TEM?ZL?K,)",0.194946,0.160442,2.384630,12.232272,76.240910,2.353352,-0.074745
222,"(KA?IT, DETERJAN TEM?ZL?K)","(DETERJAN TEM?ZL?K,)",0.128273,0.160442,1.512939,11.794634,73.513211,1.492358,-0.077775
39,"(KA?IT,)","(DETERJAN TEM?ZL?K,)",0.152183,0.160442,1.512939,9.941558,61.963423,1.488522,-0.093894
114,"(DETERJAN TEM?ZL?K,)","(KA?IT,)",0.160442,0.152183,1.512939,9.429796,61.963423,1.488522,-0.100574
253,"(MEYVE SEBZE, DETERJAN TEM?ZL?K)","(DETERJAN TEM?ZL?K,)",0.166388,0.160442,1.437020,8.636556,53.829649,1.410324,-0.109939
15,"(SÜT KAHVALTILIK,)","(DETERJAN TEM?ZL?K,)",0.602978,0.160442,4.009225,6.649043,41.441942,3.912482,-0.148619
158,"(ET TAVUK,)","(DETERJAN TEM?ZL?K,)",0.137160,0.160442,0.842139,6.139816,38.268046,0.820133,-0.163344
19,"(SÜT KAHVALTILIK,)","(KA?IT,)",0.602978,0.152183,3.417942,5.668437,37.247438,3.326178,-0.181606
179,"(GIDA, ET TAVUK)","(ET TAVUK,)",0.619369,0.137160,3.096394,4.999271,36.448381,3.011441,-0.215749
62,"(KOZMET?K,)","(DETERJAN TEM?ZL?K,)",0.147521,0.160442,0.853473,5.785427,36.059225,0.829805,-0.175440


The result is quite good but it does not meet my expectation  
Some consequents are appearing in antecedents, but it should not be there  
Let's filter the result

In [17]:
support = pd.Series(frequent_item.Support.values, index=frequent_item.Items).to_dict()

L=frequent_item.Items.values
p = [i for i in (permutations(L, 2))]
p

[(('SÜT KAHVALTILIK',), ('S?GARA',)),
 (('SÜT KAHVALTILIK',), ('KA?IT',)),
 (('SÜT KAHVALTILIK',), ('KOZMET?K',)),
 (('SÜT KAHVALTILIK',), ('GIDA',)),
 (('SÜT KAHVALTILIK',), ('MEYVE SEBZE',)),
 (('SÜT KAHVALTILIK',), ('DETERJAN TEM?ZL?K',)),
 (('SÜT KAHVALTILIK',), ('?ÇECEK',)),
 (('SÜT KAHVALTILIK',), ('ET TAVUK',)),
 (('SÜT KAHVALTILIK',), ('KA?IT', '?ÇECEK')),
 (('SÜT KAHVALTILIK',), ('KA?IT', 'MEYVE SEBZE')),
 (('SÜT KAHVALTILIK',), ('GIDA', 'ET TAVUK')),
 (('SÜT KAHVALTILIK',), ('GIDA', 'S?GARA')),
 (('SÜT KAHVALTILIK',), ('?ÇECEK', 'DETERJAN TEM?ZL?K')),
 (('SÜT KAHVALTILIK',), ('DETERJAN TEM?ZL?K', 'GIDA')),
 (('SÜT KAHVALTILIK',), ('KOZMET?K', 'SÜT KAHVALTILIK')),
 (('SÜT KAHVALTILIK',), ('KA?IT', 'SÜT KAHVALTILIK')),
 (('SÜT KAHVALTILIK',), ('MEYVE SEBZE', 'SÜT KAHVALTILIK')),
 (('SÜT KAHVALTILIK',), ('ET TAVUK', 'SÜT KAHVALTILIK')),
 (('SÜT KAHVALTILIK',), ('KA?IT', 'DETERJAN TEM?ZL?K')),
 (('SÜT KAHVALTILIK',), ('DETERJAN TEM?ZL?K', 'SÜT KAHVALTILIK')),
 (('SÜT KAHVALTILIK'

In [18]:
p[1]

(('SÜT KAHVALTILIK',), ('KA?IT',))

In [19]:
l = []
for i in range(len(p)):
    if not set(p[i][0]).issubset(p[i][1]):
        l.append(p[i])

In [20]:
l

[(('SÜT KAHVALTILIK',), ('S?GARA',)),
 (('SÜT KAHVALTILIK',), ('KA?IT',)),
 (('SÜT KAHVALTILIK',), ('KOZMET?K',)),
 (('SÜT KAHVALTILIK',), ('GIDA',)),
 (('SÜT KAHVALTILIK',), ('MEYVE SEBZE',)),
 (('SÜT KAHVALTILIK',), ('DETERJAN TEM?ZL?K',)),
 (('SÜT KAHVALTILIK',), ('?ÇECEK',)),
 (('SÜT KAHVALTILIK',), ('ET TAVUK',)),
 (('SÜT KAHVALTILIK',), ('KA?IT', '?ÇECEK')),
 (('SÜT KAHVALTILIK',), ('KA?IT', 'MEYVE SEBZE')),
 (('SÜT KAHVALTILIK',), ('GIDA', 'ET TAVUK')),
 (('SÜT KAHVALTILIK',), ('GIDA', 'S?GARA')),
 (('SÜT KAHVALTILIK',), ('?ÇECEK', 'DETERJAN TEM?ZL?K')),
 (('SÜT KAHVALTILIK',), ('DETERJAN TEM?ZL?K', 'GIDA')),
 (('SÜT KAHVALTILIK',), ('KA?IT', 'DETERJAN TEM?ZL?K')),
 (('SÜT KAHVALTILIK',), ('?ÇECEK', 'GIDA')),
 (('SÜT KAHVALTILIK',), ('MEYVE SEBZE', 'GIDA')),
 (('SÜT KAHVALTILIK',), ('MEYVE SEBZE', 'DETERJAN TEM?ZL?K')),
 (('SÜT KAHVALTILIK',), ('?ÇECEK', 'MEYVE SEBZE')),
 (('SÜT KAHVALTILIK',), ('KA?IT', 'GIDA')),
 (('SÜT KAHVALTILIK',), ('KOZMET?K', 'GIDA')),
 (('SÜT KAHVALTILI

In [ ]:
result['inBasket'] = 0
for i in range(len(result)):
    if str(result['consequents'][i])[1:-2] in str(result['antecedents'][i]):
        result['inBasket'][i] = 1
    else:
        result['inBasket'][i] = 0
final_result = result[result['inBasket'] == 0]
final_result.sort_values(by='Lift', ascending=False)

#### Apply Apriori for items

In [21]:
selected_col = ['FICHENO','ITEMNAME']
data = market_sale[selected_col]

In [22]:
top1000_count = data['ITEMNAME'].value_counts().rename('top1000_count')
data = data.merge(top1000_count.to_frame(),
                                left_on='ITEMNAME',
                                right_index=True)

There are a lot of items and goods in database  
My computer can not be adapted  
So I only choose items appearing over 1000 times to apply apriori

In [23]:
data = data[data['top1000_count'] > 1000]
data.head()

,FICHENO,ITEMNAME,top1000_count
1,18105,TOZ SEKER,5242
5,18127,TOZ SEKER,5242
7,18354,TOZ SEKER,5242
8,18106,TOZ SEKER,5242
256,17549,TOZ SEKER,5242


In [24]:
data = data.drop('top1000_count', axis = 1)

In [25]:
transaction = pd.crosstab(index = data['FICHENO'], columns = data['ITEMNAME'])

In [26]:
transaction

ITEMNAME,AL GUNDUZ TRAKYA BEYAZ PEYNIR,BILLUR TUZ 750 GR.,BURDAN 6 LI YUMURTA,CARLISTON,CERI DOMATES,COCA COLA 1 LT. PET SISE,"COCA COLA 2,5 LT. EKO BOY",DANA GULAS,DANA KIYMALIK KG.,DERE OTU,...,ULKER LAVIVA DOLGULU CIKOLATA,WINSTON LIGHTS BOX,YAHYAOGLU 15 LI YUMURTA,YAHYAOGLU 30 LU BUYUK BOY YUMURTA,YAHYAOGLU 6 LI YUMURTA,YERLI BADEM,YERLI BALDO PIRINC,YERLI MUZ,YESIL MERCIMEK,YESIL SOGAN
FICHENO,,,,,,,,,,,,,,,,,,,,,
15560,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15566,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351111,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
351114,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
351118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
transaction.shape

(73604, 76)

In [28]:
frequent_item = apriori(transaction, min_support = 0.001, max_length = 3)
frequent_item

,Items,Support
0,"(LIMON FILE,)",0.027893
1,"(F SAFF ISLAK MENDIL 80 AD,)",0.018192
2,"(ISPANAK KG,)",0.018627
3,"(MANTAR PAKET,)",0.014714
4,"(OSMANCIK PIRINC KG.,)",0.055717
...,...,...
1439,"(KIVIRCIK, MAYDANOZ, HAVUC)",0.003899
1440,"(KIVIRCIK, S?H?RL? ELLER C?G KÖFTE 200GR, HAVUC)",0.001100
1441,"(PATATES, YESIL SOGAN, HAVUC)",0.001223
1442,"(SOGAN, DOMATES, CARLISTON)",0.002269


In [29]:
result = associate_rule(frequent_item, min_threshold=0.5)

In [30]:
result.sort_values(by='Lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
9,"(SOGAN, ELMA IRI)","(ELMA IRI,)",0.001698,0.016466,0.001087,0.640000,38.866799,0.001059,2.732038
20,"(LIMON, ELMA IRI)","(ELMA IRI,)",0.001821,0.016466,0.001128,0.619403,37.615955,0.001098,2.584186
72,"(KIVIRCIK, ELMA IRI)","(ELMA IRI,)",0.001943,0.016466,0.001155,0.594406,36.097879,0.001123,2.424919
49,"(TOZ SEKER, NOHUT BEYAZ)","(NOHUT BEYAZ,)",0.004986,0.020896,0.002622,0.525886,25.167283,0.002518,2.065122
44,"(PATLICAN, SALATALIK)","(SALATALIK,)",0.002187,0.026384,0.001318,0.602484,22.834844,0.001260,2.449252
...,...,...,...,...,...,...,...,...,...
29,"(MUZ, ELMA GOLDEN)","(PORTAKAL,)",0.003560,0.076232,0.001807,0.507634,6.659038,0.001536,1.876179
4,"(SALATALIK, HAVUC)","(DOMATES,)",0.003750,0.081762,0.002038,0.543478,6.647088,0.001731,2.011379
58,"(SOGAN, F NEFFIS SÜT YARIM YA?LI 1 LT)","(PATATES,)",0.003247,0.077686,0.001644,0.506276,6.516955,0.001392,1.868077
12,"(KIVIRCIK, CARLISTON)","(DOMATES,)",0.003777,0.081762,0.001970,0.521583,6.379291,0.001661,1.919325


In [31]:
result['inBasket'] = 0
for i in range(len(result)):
    if str(result['consequents'][i])[1:-2] in str(result['antecedents'][i]):
        result['inBasket'][i] = 1
    else:
        result['inBasket'][i] = 0

C:\Users\vient\Anaconda3\envs\plot_env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vient\Anaconda3\envs\plot_env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [32]:
result.sort_values(by='Lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection,inBasket
9,"(SOGAN, ELMA IRI)","(ELMA IRI,)",0.001698,0.016466,0.001087,0.640000,38.866799,0.001059,2.732038,1
20,"(LIMON, ELMA IRI)","(ELMA IRI,)",0.001821,0.016466,0.001128,0.619403,37.615955,0.001098,2.584186,1
72,"(KIVIRCIK, ELMA IRI)","(ELMA IRI,)",0.001943,0.016466,0.001155,0.594406,36.097879,0.001123,2.424919,1
49,"(TOZ SEKER, NOHUT BEYAZ)","(NOHUT BEYAZ,)",0.004986,0.020896,0.002622,0.525886,25.167283,0.002518,2.065122,1
44,"(PATLICAN, SALATALIK)","(SALATALIK,)",0.002187,0.026384,0.001318,0.602484,22.834844,0.001260,2.449252,1
...,...,...,...,...,...,...,...,...,...,...
29,"(MUZ, ELMA GOLDEN)","(PORTAKAL,)",0.003560,0.076232,0.001807,0.507634,6.659038,0.001536,1.876179,0
4,"(SALATALIK, HAVUC)","(DOMATES,)",0.003750,0.081762,0.002038,0.543478,6.647088,0.001731,2.011379,0
58,"(SOGAN, F NEFFIS SÜT YARIM YA?LI 1 LT)","(PATATES,)",0.003247,0.077686,0.001644,0.506276,6.516955,0.001392,1.868077,0
12,"(KIVIRCIK, CARLISTON)","(DOMATES,)",0.003777,0.081762,0.001970,0.521583,6.379291,0.001661,1.919325,0


In [33]:
final_result = result[result['inBasket'] == 0]
final_result.sort_values(by='Lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection,inBasket
15,"(KABAK, CARLISTON)","(PATLICAN,)",0.003356,0.024170,0.001821,0.542510,22.445708,0.001739,2.133009,0
39,"(PILAVLIK BULGUR, F SAFF ISLAK MENDIL 63 AD)","(OSMANCIK PIRINC KG.,)",0.001087,0.055717,0.001250,1.150000,20.639990,0.001189,-6.295219,0
63,"(CARLISTON, DERE OTU)","(MAYDANOZ,)",0.001535,0.051736,0.001196,0.778761,15.052502,0.001116,4.286152,0
59,"(KIVIRCIK, DERE OTU)","(MAYDANOZ,)",0.003247,0.051736,0.002296,0.707113,13.667632,0.002128,3.237643,0
60,"(YESIL SOGAN, DERE OTU)","(MAYDANOZ,)",0.003220,0.051736,0.002269,0.704641,13.619859,0.002102,3.210550,0
18,"(YESIL SOGAN, LIMON FILE)","(KIVIRCIK,)",0.002350,0.053326,0.001630,0.693642,13.007592,0.001505,3.090087,0
45,"(DOMATES, DERE OTU)","(MAYDANOZ,)",0.002785,0.051736,0.001861,0.668293,12.917283,0.001717,2.858736,0
11,"(HAVUC, DERE OTU)","(MAYDANOZ,)",0.002894,0.051736,0.001875,0.647887,12.522873,0.001725,2.693069,0
26,"(YESIL SOGAN, F NEFFIS SÜT YARIM YA?LI 1 LT)","(MAYDANOZ,)",0.001712,0.051736,0.001100,0.642857,12.425645,0.001012,2.655138,0
28,"(YESIL SOGAN, S?H?RL? ELLER C?G KÖFTE 200GR)","(KIVIRCIK,)",0.002255,0.053326,0.001427,0.632530,11.861592,0.001306,2.576195,0


As we can see, Customers offen buy Kabak, carliston and then buy Patlican  
When customers buy Dere Otu or Yesil Sogan, they also buy Maydanoz

#### After basket analys, we continue to divide custormer segments  
In this post, I try to use 2 ways for customer segments:  
First way is Customer Lifetime Value https://blog.hubspot.com/service/how-to-calculate-customer-lifetime-value  
Second way is K-Means Clustering from Sklearn

In [34]:
from datetime import datetime

In [35]:
market_sale['DATE_'] = pd.to_datetime(market_sale['DATE_'])

In [36]:
market_sale1 = market_sale[market_sale['DATE_'] < '2017-04-01']

In [37]:
select_clv = ['DATE_', 'FICHENO', 'AMOUNT', 'LINENETTOTAL', 'CLIENTCODE','GENDER']

In [38]:
data_clv = market_sale1[select_clv]
data_clv = data_clv.dropna(axis = 0)
data_clv.head()

,DATE_,FICHENO,AMOUNT,LINENETTOTAL,CLIENTCODE,GENDER
2,2017-03-01,18350,1.0,0.40,656969,E
3,2017-03-01,18350,1.0,0.40,656969,E
7,2017-03-01,18354,2.0,5.30,144939,E
9,2017-03-01,18365,1.0,2.85,784782,K
12,2017-02-01,15671,1.0,3.45,600916,E


In [39]:
clv = data_clv.groupby('CLIENTCODE').agg({'DATE_': lambda date: (date.max() - date.min()).days,
                                          'FICHENO':lambda transaction: len(transaction),
                                          'AMOUNT': lambda num: num.sum(),'LINENETTOTAL': lambda total: total.sum()})

In [40]:
clv_col = ['time_period', 'number_transaction', 'number_buy', 'total_sale']
clv.columns = clv_col
clv1 = clv.copy()

In [41]:
clv1['average_sale_value'] = clv1['total_sale']/clv1['number_buy']
clv1['time_period'] = clv1['time_period']
churn_rate=clv1[clv1["number_transaction"]<=1].shape[0]/clv1.shape[0]
clv1['lifetime'] = clv1['average_sale_value']/churn_rate

In [42]:
clv1['profit_margin'] = clv1['average_sale_value']*0.05

In [43]:
clv1['CLTV'] = clv1['profit_margin']*clv1['lifetime']
clv1

,time_period,number_transaction,number_buy,total_sale,average_sale_value,lifetime,profit_margin,CLTV
CLIENTCODE,,,,,,,,
1.0,0,7,9.240,20.09,2.174242,10.296384,0.108712,1.119342
2.0,25,11,14.320,57.44,4.011173,18.995388,0.200559,3.809690
3.0,38,21,32.587,103.13,3.164759,14.987093,0.158238,2.371527
4.0,36,11,18.775,27.87,1.484421,7.029651,0.074221,0.521748
5.0,58,2,2.000,7.75,3.875000,18.350524,0.193750,3.555414
...,...,...,...,...,...,...,...,...
999894,0,2,4.104,23.43,5.709064,27.035953,0.285453,7.717500
999903,42,10,11.004,23.31,2.118321,10.031559,0.105916,1.062503
999955,0,1,0.425,1.11,2.611765,12.368322,0.130588,1.615157


In [44]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(clv1[["CLTV"]])
clv1["SCALED_CLTV"] = scaler.transform(clv1[["CLTV"]])
clv1["segment"] = pd.qcut(clv1["SCALED_CLTV"], 4, labels=['D', 'C', 'B', 'A'])
clv1.head()

,time_period,number_transaction,number_buy,total_sale,average_sale_value,lifetime,profit_margin,CLTV,SCALED_CLTV,segment
CLIENTCODE,,,,,,,,,,
1.0,0,7,9.240,20.09,2.174242,10.296384,0.108712,1.119342,0.000009,C
2.0,25,11,14.320,57.44,4.011173,18.995388,0.200559,3.809690,0.000031,B
3.0,38,21,32.587,103.13,3.164759,14.987093,0.158238,2.371527,0.000019,B
4.0,36,11,18.775,27.87,1.484421,7.029651,0.074221,0.521748,0.000004,D
5.0,58,2,2.000,7.75,3.875000,18.350524,0.193750,3.555414,0.000029,B


Customers who have time_period = 0 are Current Customers, they come to market 1 times during considering period  
So I think they should be belong to a separate segment called Current

In [45]:
segment = []
for i in range(len(clv1)):
    if clv1['time_period'][i] == 0:
        segment.append('Current')
    else:
        segment.append(clv1['segment'][i])
clv1['segment'] = segment

In [46]:
clv1

,time_period,number_transaction,number_buy,total_sale,average_sale_value,lifetime,profit_margin,CLTV,SCALED_CLTV,segment
CLIENTCODE,,,,,,,,,,
1.0,0,7,9.240,20.09,2.174242,10.296384,0.108712,1.119342,0.000009,Current
2.0,25,11,14.320,57.44,4.011173,18.995388,0.200559,3.809690,0.000031,B
3.0,38,21,32.587,103.13,3.164759,14.987093,0.158238,2.371527,0.000019,B
4.0,36,11,18.775,27.87,1.484421,7.029651,0.074221,0.521748,0.000004,D
5.0,58,2,2.000,7.75,3.875000,18.350524,0.193750,3.555414,0.000029,B
...,...,...,...,...,...,...,...,...,...,...
999894,0,2,4.104,23.43,5.709064,27.035953,0.285453,7.717500,0.000062,Current
999903,42,10,11.004,23.31,2.118321,10.031559,0.105916,1.062503,0.000009,C
999955,0,1,0.425,1.11,2.611765,12.368322,0.130588,1.615157,0.000013,Current


In [47]:
clv1['segment'].value_counts()

Current    39067
B          10939
C           9929
A           7796
D           6443
Name: segment, dtype: int64

In [48]:
from sklearn.cluster import KMeans

In [49]:
clv

,time_period,number_transaction,number_buy,total_sale
CLIENTCODE,,,,
1.0,0,7,9.240,20.09
2.0,25,11,14.320,57.44
3.0,38,21,32.587,103.13
4.0,36,11,18.775,27.87
5.0,58,2,2.000,7.75
...,...,...,...,...
999894,0,2,4.104,23.43
999903,42,10,11.004,23.31
999955,0,1,0.425,1.11


In [53]:
scaled = MinMaxScaler()
clv_scaled = scaled.fit_transform(clv)
clv_scaled

array([[0.00000000e+00, 4.80000000e-02, 1.43781621e-02, 4.68436003e-03],
       [2.84090909e-01, 8.00000000e-02, 2.23130956e-02, 1.34018910e-02],
       [4.31818182e-01, 1.60000000e-01, 5.08460545e-02, 2.40659872e-02],
       ...,
       [0.00000000e+00, 0.00000000e+00, 6.09177969e-04, 2.54407196e-04],
       [0.00000000e+00, 0.00000000e+00, 7.60691464e-04, 3.22326916e-03],
       [6.13636364e-01, 4.00000000e-02, 1.08792937e-02, 4.41594876e-03]])

In [55]:
kmean_cl = KMeans(n_clusters=4,random_state = 0)

In [58]:
kmean_cl.fit(clv_scaled)

KMeans(n_clusters=4, random_state=0)

In [59]:
kmean_cl.labels_

array([1, 2, 3, ..., 1, 1, 0])

In [60]:
clv['Segment'] = kmean_cl.labels_

In [61]:
clv

,time_period,number_transaction,number_buy,total_sale,Segment
CLIENTCODE,,,,,
1.0,0,7,9.240,20.09,1
2.0,25,11,14.320,57.44,2
3.0,38,21,32.587,103.13,3
4.0,36,11,18.775,27.87,3
5.0,58,2,2.000,7.75,0
...,...,...,...,...,...
999894,0,2,4.104,23.43,1
999903,42,10,11.004,23.31,3
999955,0,1,0.425,1.11,1


In [62]:
clv1

,time_period,number_transaction,number_buy,total_sale,average_sale_value,lifetime,profit_margin,CLTV,SCALED_CLTV,segment
CLIENTCODE,,,,,,,,,,
1.0,0,7,9.240,20.09,2.174242,10.296384,0.108712,1.119342,0.000009,Current
2.0,25,11,14.320,57.44,4.011173,18.995388,0.200559,3.809690,0.000031,B
3.0,38,21,32.587,103.13,3.164759,14.987093,0.158238,2.371527,0.000019,B
4.0,36,11,18.775,27.87,1.484421,7.029651,0.074221,0.521748,0.000004,D
5.0,58,2,2.000,7.75,3.875000,18.350524,0.193750,3.555414,0.000029,B
...,...,...,...,...,...,...,...,...,...,...
999894,0,2,4.104,23.43,5.709064,27.035953,0.285453,7.717500,0.000062,Current
999903,42,10,11.004,23.31,2.118321,10.031559,0.105916,1.062503,0.000009,C
999955,0,1,0.425,1.11,2.611765,12.368322,0.130588,1.615157,0.000013,Current
